<a href="https://colab.research.google.com/github/illhammm/Sign_preprocessing/blob/woork_with_open_cv/more_clean_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Установка необходимых библиотек*

In [1]:
!pip install pyarrow
!pip install pandas
!pip install opencv-python
!pip install opencv-python

*Импорт библиотек*

In [2]:
import pandas as pd
import pyarrow.parquet as pq
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2

*Чтение DataFrame из .pkl файла* (**если файл не найден, загрузка из Google Colab**)

In [3]:
try:
    df = pd.read_pickle('train-00000-of-00001-a16b0610cb895cd7.pkl')
except FileNotFoundError:
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    table = pq.read_table(file_name)
    df = table.to_pandas()
    df.to_pickle('train-00000-of-00001-a16b0610cb895cd7.pkl')

Saving train-00000-of-00001-a16b0610cb895cd7.parquet to train-00000-of-00001-a16b0610cb895cd7.parquet


*Вывод информации о DataFrame*

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  504 non-null    int64 
 1   image     504 non-null    object
 2   width     504 non-null    int32 
 3   height    504 non-null    int32 
 4   objects   504 non-null    object
dtypes: int32(2), int64(1), object(2)
memory usage: 15.9+ KB
None


*Вывод первых несколько строк DataFrame*

In [6]:
print(df.head())

   image_id                                              image  width  height  \
0       184  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    640     640   
1       403  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    640     640   
2       393  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    640     640   
3       317  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    640     640   
4       304  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    640     640   

                                             objects  
0  {'id': [184], 'area': [39601], 'bbox': [[200.0...  
1  {'id': [403], 'area': [81840], 'bbox': [[56.0,...  
2  {'id': [393], 'area': [91178], 'bbox': [[87.0,...  
3  {'id': [317], 'area': [30397], 'bbox': [[237.0...  
4  {'id': [304], 'area': [92757], 'bbox': [[32.0,...  


**Вывод названий всех столбцов в DataFrame**, *чтобы знать название шапки таблицы строки с изображениями*

In [7]:
print(df.columns)

Index(['image_id', 'image', 'width', 'height', 'objects'], dtype='object')


*Узнаем тип данных изображений в столбце*

In [8]:
data_type = type(df.iloc[0, 1])
print(f"Тип данных во втором столбце: {data_type}")

Тип данных во втором столбце: <class 'dict'>


*Функция для преобразования словаря с изображением в формат OpenCV*

In [9]:
def convert_dict_to_opencv(image_dict):
    image_data = image_dict.get('image_data', None)

    if image_data is not None:
        image_array = np.array(image_data)
        opencv_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
        return opencv_image
    else:
        return None

Применение функции к столбцу '**image**' и создание нового столбца '**image_opencv**'

In [11]:
df['image_opencv'] = df['image'].apply(convert_dict_to_opencv)

*Проверим то, что столбец добавился*

In [12]:
print(df.columns)

Index(['image_id', 'image', 'width', 'height', 'objects', 'image_opencv'], dtype='object')


**Функция для отображения изображения из словаря**

In [13]:
def show_image_from_dict(image_dict):
    image_data = image_dict.get('image_data', None)

    if image_data is not None:
        image = Image.fromarray(image_data)
        plt.imshow(image)
        plt.axis('off')
        plt.show()
    else:
        print("Данные изображения не найдены в словаре.")

*Вызываем функцию для отображения первого изображения в столбце* '**image**'

In [14]:
show_image_from_dict(df['image'].iloc[1])

Данные изображения не найдены в словаре.


*Функция для проверки наличия лица на изображении*

In [15]:
def has_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return len(faces) > 0

*Узнаем тип данных столбца image_opencv*

In [16]:
data_types = df['image_opencv'].dtypes
print(f"Типы данных в столбце 'image_opencv':\n{data_types}")

Типы данных в столбце 'image_opencv':
object


*Применение функции к столбцу 'image_opencv' и создание нового столбца 'has_face'*

*Добавим проверку на None в функцию **has_face**

In [20]:
def has_face(image):
    if image is not None:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        return len(faces) > 0
    else:
        return False

*Инициализация каскада для распознавания лиц*

In [21]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')